<a href="https://colab.research.google.com/github/DorZarka/CV_Final_Project/blob/master/ObjectMeasurment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
print("Welcome Daniel!")

Welcome Daniel!


In [ ]:
print("Welcome is wrriten with E at the end of the word")

Welcome is wrriten with E at the end of the word
